In [1]:
import cv2
import numpy 
import copy
import collections

img_size = [601,1001]
img = numpy.zeros([601,1001,3])
print(img.shape)

img[:,:,0] = numpy.ones(img_size)*240.0
img[:,:,1] = numpy.ones(img_size)*240.0
img[:,:,2] = numpy.ones(img_size)*240.0

img = cv2.circle(img,(2*380,2*260), 30, (0,0,0), -1)
img = cv2.rectangle(img,(2*100,2*225),(2*200,2*135),(0,0,0),-1)
img = cv2.ellipse(img,(2*280,2*240),(2*30,2*12),0,0,359,(0,0,0),-1)

pts = numpy.array([[2*250, 2*112],[2*326,2*104], [2*340, 2*180], [2*384, 2*104],[2*346,2*30],[2*300,2*30]], numpy.int32)
img = cv2.fillPoly(img,[pts],(0,0,0))

for x in range(0,150):
    for y in range(0,250):
        for channel in range(3):
            res = img[4*x:4*(x+1),4*y:4*(y+1),channel].sum()
            if res != 3840:
                img[4*x:4*(x+1),4*y:4*(y+1),channel] = 0#([ [0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0] ])
            

img = cv2.flip( img, 0 )
cv2.imwrite('Robot_Map.jpg', img)

(601, 1001, 3)


True

In [3]:
def Take_Starting_Node():
    
    img = cv2.imread('Robot_Map.jpg')
    gx = 4*140
    gy = 4*120
    while img[600-gy][gx][0] != 240:
        X = ((input("Give a Starting Node x in range(0,250) and y in range(0,150)   ")).split(','))
        #print(X)
        gx = 4*int(X[0])
        gy = 4*int(X[1])
    
        #gx,gy = Goal_Node
        if img[600-gy][gx][0] == 240:
            print('Given Starting Node is acceptable')
            img = cv2.rectangle(img,(gx,600-(gy + 3)),(gx+3,600-gy),(0,0,255),-1)

            cv2.imwrite('Robot_Map.jpg', img)
            break
        else:
            print('Given Node is in Obstacle Space, please give a new node')
          
    return(int(X[0]),int(X[1]))


In [4]:
def Take_Goal_Node():
    img = cv2.imread('Robot_Map.jpg')
    
    gx = 4*140
    gy = 4*120
    while img[600-gy][gx][0] != 240:
        X = ((input("Give a Goal Node x in range(0,250) and y in range(0,150)   ")).split(','))
        #print(X)
        gx = 4*int(X[0])
        gy = 4*int(X[1])
        
#         Goal_Node = (600-gy,gx)
    
        if img[600-gy][gx][0] in range(237,243):
            print('Given Goal Node is acceptable')
            img = cv2.rectangle(img,(gx,600-(gy + 3)),(gx+3,600-gy),(0,255,0),-1)

            cv2.imwrite('Robot_Map.jpg', img)
            break
        else:
            print(img[600-gy][gx])
            print('Given Node is in Obstacle Space, please give a new node')
     
    return(int(X[0]),int(X[1]))

In [6]:
def Mark_Read(x,y):
    
    x = 4*x
    y = 4*y
    img = cv2.imread('Robot_Map.jpg')
    img = cv2.rectangle(img,(x,600-(y + 2)),(x+2,600-y),(100,100,100),-1)
    cv2.imwrite('Robot_Map.jpg', img)
    cv2.imshow('Robot_Map.jpg', img)

In [7]:
class Node:

    def __init__(self, child, parent=None,cost = 999999):
        self.child = child
        self.parent = parent
        self.cost = cost
                     

In [8]:
def Node_Actions(node):
    
    (i,j) = node
    directions = {'R':((i+1, j),1),
                  'L':((i-1, j),1),
                  'D':((i, j-1),1),
                  'U':((i, j+1),1),
                  'RU':((i+1,j+1),1.414),
                  'RD':((i+1,j-1),1.414),
                  'LU':((i-1,j+1),1.414),
                  'LD':((i-1,j-1),1.414)}
    #print(directions)
    
    valid_direc = {}
    for i in directions.items():
        d,((x,y),action_cost) = i
        if x in range(0,251) and y in range(0,151):
            X = 4*x
            Y = 4*y
            if img[600-Y][X][0] == 240:
                valid_direc.update({i})
    
    return(valid_direc)
#     for i in valid_direc.items():
#         print(i)
    

In [9]:
def Solve_Dijkstra(Start,Goal):
    
    start_node = Node(Start,None,0)
    #Goal_Node = Node(Goal)
    flag = True
    queue = collections.deque([start_node])
    seen  = {}
    seen[str((queue[0].child))] = start_node
    
    while queue and flag:
            
        node = queue.pop()
            
        A = Node_Actions(node.child)

        for i in A.items():
            d,((x,y),action_cost) = i
            #new_node = Node((x,y))
            
            if str((x,y)) not in seen and (x,y) == Goal:
                
                new_node = Node((x,y))
                Goal_Node = Node((x,y))
                Goal_Node.parent = node
                new_node.cost = node.cost + action_cost
                flag = False
        
            if str((x,y)) not in seen:
                new_node = Node((x,y))
                
                queue.appendleft(new_node)
                seen[(str(new_node.child))]=new_node
                
                new_node.parent = node
                new_node.cost = node.cost + action_cost
                #print(new_node.cost,(x,y))
                
                if new_node.child != Goal:
                    m,n = new_node.child
                    Mark_Read(m,n)
                    cv2.waitKey(1)
                
            else:
                if seen[str((x,y))].cost > node.cost + action_cost:
                    seen[str((x,y))].cost = node.cost + action_cost
                    seen[str((x,y))].parent = node
        
    
    while Goal_Node.child != Start and Goal_Node.parent:
        Goal_Node = Goal_Node.parent
        p,q = Goal_Node.child
        P = 4*p
        Q = 4*q
        if (p,q) != Start:
            img = cv2.imread('Robot_Map.jpg')
            img = cv2.rectangle(img,(P,600-(Q + 2)),(P+2,600-Q),(255,255,255),-1)
            cv2.imwrite('Robot_Map.jpg', img)
            cv2.imshow('Robot_Map.jpg', img)
            cv2.waitKey(1)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows
                     

In [10]:
Start = Take_Starting_Node()
Goal = Take_Goal_Node()

Give a Starting Node x in range(0,250) and y in range(0,150)   10,10
Given Starting Node is acceptable
Give a Goal Node x in range(0,250) and y in range(0,150)   50,50
Given Goal Node is acceptable


In [ ]:
Solve_Dijkstra(Start,Goal)